## SF crime data analysis and modeling

In [2]:
from csv import reader
from pyspark.sql import Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import warnings

import os
os.environ["PYSPARK_PYTHON"] = "python3"


In [3]:

import urllib.request
# urllib.request.urlretrieve("https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD", "/tmp/myxxxx.csv")
# dbutils.fs.mv("file:/tmp/myxxxx.csv", "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv")
# display(dbutils.fs.ls("dbfs:/laioffer/spark_hw1/data/"))

# https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD


In [4]:
data_path = "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv"
# use this file name later

In [5]:
# data preprocessing 
# read data from the data storage
# please upload your data into databricks community at first. 
crime_data_lines = sc.textFile(data_path)
#prepare data 
df_crimes = crime_data_lines.map(lambda line: [x.strip('"') for x in next(reader([line]))])
#get header
header = df_crimes.first()
print(header)

#remove the first line of data
crimes = df_crimes.filter(lambda x: x != header)

#get the first line of data
#display(crimes.take(3))

#get the total number of data 
print(crimes.count())

# ['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId']
# 2215024



['PdId', 'IncidntNum', 'Incident Code', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'location', 'SF Find Neighborhoods 2 2', 'Current Police Districts 2 2', 'Current Supervisor Districts 2 2', 'Analysis Neighborhoods 2 2', 'DELETE - Fire Prevention Districts 2 2', 'DELETE - Police Districts 2 2', 'DELETE - Supervisor Districts 2 2', 'DELETE - Zip Codes 2 2', 'DELETE - Neighborhoods 2 2', 'DELETE - 2017 Fix It Zones 2 2', 'Civic Center Harm Reduction Project Boundary 2 2', 'Fix It Zones as of 2017-11-06 2 2', 'DELETE - HSOC Zones 2 2', 'Fix It Zones as of 2018-02-07 2 2', 'CBD, BID and GBD Boundaries as of 2017 2 2', 'Areas of Vulnerability, 2016 2 2', 'Central Market/Tenderloin Boundary 2 2', 'Central Market/Tenderloin Boundary Polygon - Updated 2 2', 'HSOC Zones as of 2018-06-05 2 2', 'OWED Public Spaces 2 2', 'Neighborhoods 2']
2160953

### Solove  big data issues via Spark
approach 1: use RDD (not recommend)  
approach 2: use Dataframe, register the RDD to a dataframe (recommend for DE)  
approach 3: use SQL (recomend for data analysis or DS， 基础比较差的同学)  
***note***: you only need to choose one of approaches as introduced above

In [7]:

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("crime analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df_opt1 = spark.read.format("csv").option("header", "true").load(data_path)
display(df_opt1)
df_opt1.createOrReplaceTempView("sf_crime")

## helper function to transform the date, choose your way to do it. 
 
# refer: https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-functions-datetime.html
# 方法1 使用系统自带udf
# from pyspark.sql.functions import to_date, to_timestamp, hour
# df_opt1 = df_opt1.withColumn('Date', to_date(df_opt1.OccurredOn, "MM/dd/yy"))
# df_opt1 = df_opt1.withColumn('Time', to_timestamp(df_opt1.OccurredOn, "MM/dd/yy HH:mm"))
# df_opt1 = df_opt1.withColumn('Hour', hour(df_opt1['Time']))
# df_opt1 = df_opt1.withColumn("DayOfWeek", date_format(df_opt1.Date, "EEEE"))

## 方法2 手工写udf 
#from pyspark.sql.functions import col, udf
#from pyspark.sql.functions import expr
#from pyspark.sql.functions import from_unixtime

#date_func =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())
#month_func = udf (lambda x: datetime.strptime(x, '%m/%d/%Y').strftime('%Y/%m'), StringType())

#df = df_opt1.withColumn('month_year', month_func(col('Date')))\
#           .withColumn('Date_time', date_func(col('Date')))

## 方法3 手工在sql 里面
# select Date, substring(Date,7) as Year, substring(Date,1,2) as Month from sf_crime


## 方法4: 使用系统自带
# from pyspark.sql.functions import *
# df_update = df_opt1.withColumn("Date", to_date(col("Date"), "MM/dd/yyyy")) ##change datetype from string to date
# df_update.createOrReplaceTempView("sf_crime")
# crimeYearMonth = spark.sql("SELECT Year(Date) AS Year, Month(Date) AS Month, FROM sf_crime")

PdId,IncidntNum,Incident Code,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,location,SF Find Neighborhoods 2 2,Current Police Districts 2 2,Current Supervisor Districts 2 2,Analysis Neighborhoods 2 2,DELETE - Fire Prevention Districts 2 2,DELETE - Police Districts 2 2,DELETE - Supervisor Districts 2 2,DELETE - Zip Codes 2 2,DELETE - Neighborhoods 2 2,DELETE - 2017 Fix It Zones 2 2,Civic Center Harm Reduction Project Boundary 2 2,Fix It Zones as of 2017-11-06 2 2,DELETE - HSOC Zones 2 2,Fix It Zones as of 2018-02-07 2 2,"CBD, BID and GBD Boundaries as of 2017 2 2","Areas of Vulnerability, 2016 2 2",Central Market/Tenderloin Boundary 2 2,Central Market/Tenderloin Boundary Polygon - Updated 2 2,HSOC Zones as of 2018-06-05 2 2,OWED Public Spaces 2 2,Neighborhoods 2
7121491514040,071214915,14040,OTHER OFFENSES,INDECENT EXPOSURE,Tuesday,11/27/2007,13:53,NORTHERN,"ARREST, CITED",600 Block of TURK ST,-122.419820628063,37.782008780606,POINT (-122.41982062806301 37.782008780606),21,4,10,36,7,9,9,28852,36,null,1,null,1,null,8,2,1,1,1,null,21
7036663851040,070366638,51040,NON-CRIMINAL,AIDED CASE,Tuesday,03/20/2007,23:10,TARAVAL,NONE,2400 Block of 15TH AV,-122.47122106943,37.7422967951105,POINT (-122.47122106942999 37.7422967951105),46,10,8,41,1,8,4,29491,40,null,null,null,null,null,null,1,null,null,null,null,46
4059322571000,040593225,71000,NON-CRIMINAL,LOST PROPERTY,Thursday,05/20/2004,12:00,TARAVAL,NONE,19TH AV / WAWONA ST,-122.475287886017,37.7374527165306,POINT (-122.475287886017 37.7374527165306),null,10,8,41,1,8,4,29491,40,null,null,null,null,null,null,1,null,null,null,null,null
3085157264070,030851572,64070,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Thursday,07/17/2003,17:35,INGLESIDE,PROSECUTED BY OUTSIDE AGENCY,100 Block of 29TH ST,-122.423538343437,37.743834466729,POINT (-122.42353834343699 37.743834466729),84,9,5,22,2,7,5,28859,22,null,null,null,null,null,null,1,null,null,null,null,84
13067727606304,130677276,06304,LARCENY/THEFT,GRAND THEFT FROM A BUILDING,Thursday,08/15/2013,04:42,SOUTHERN,NONE,200 Block of KANSAS ST,-122.403631162228,37.7666581536244,POINT (-122.40363116222801 37.7666581536244),33,1,9,4,8,2,8,28853,20,null,null,null,null,null,null,1,null,null,null,null,33
11056206728135,110562067,28135,OTHER OFFENSES,HARASSING PHONE CALLS,Wednesday,07/13/2011,17:00,CENTRAL,NONE,600 Block of STOCKTON ST,-122.407354509443,37.7917926375029,POINT (-122.407354509443 37.7917926375029),104,6,3,8,5,1,10,28857,6,null,null,null,null,null,null,2,null,null,null,null,104
11081732316710,110817323,16710,DRUG/NARCOTIC,POSSESSION OF NARCOTICS PARAPHERNALIA,Monday,10/10/2011,14:26,TENDERLOIN,"ARREST, BOOKED",300 Block of HYDE ST,-122.415910622743,37.7839999167561,POINT (-122.415910622743 37.7839999167561),20,5,10,36,7,10,9,28858,36,17,null,18,null,18,6,2,1,1,null,null,20
12086972806153,120869728,06153,LARCENY/THEFT,GRAND THEFT FROM PERSON,Sunday,10/28/2012,13:30,MISSION,NONE,DOLORES ST / 17TH ST,-122.426228446916,37.7630312947078,POINT (-122.42622844691601 37.7630312947078),37,3,5,5,8,4,5,28859,3,null,null,null,null,null,null,1,null,null,5,null,37
5060574104022,050605741,04022,ASSAULT,ATTEMPTED HOMICIDE WITH A KNIFE,Tuesday,05/31/2005,08:00,INGLESIDE,"ARREST, BOOKED",100 Block of OCEAN AV,-122.43934468789,37.723624467748,POINT (-122.43934468789 37.723624467748),94,9,1,28,9,7,6,28861,25,null,null,null,null,null,null,1,null,null,null,null,94
8124763319057,081247633,19057,ASSAULT,THREATS AGAINST LIFE,Friday,11/21/2008,17:15,MISSION,NONE,300 Block of GUERRERO ST,-122.424061577036,37.7655856465344,POINT (-122.424061577036 37.7655856465344),37,3,5,20,8,4,5,28853,19,null,null,null,null,null,null,2,null,null,null,null,37


#### Q1 question (OLAP): 
#####Write a Spark program that counts the number of crimes for different category.

Below are some example codes to demonstrate the way to use Spark RDD, DF, and SQL to work with big data. You can follow this example to finish other questions.

In [9]:
q1_result = df_opt1.groupBy('category').count().orderBy('count', ascending=False)
display(q1_result)

category,count
LARCENY/THEFT,477975
OTHER OFFENSES,303027
NON-CRIMINAL,236937
ASSAULT,191384
VEHICLE THEFT,126228
DRUG/NARCOTIC,117875
VANDALISM,114718
WARRANTS,99821
BURGLARY,91067
SUSPICIOUS OCC,79740


In [10]:
#Spark SQL based
crimeCategory = spark.sql("SELECT  category, COUNT(*) AS Count FROM sf_crime GROUP BY category ORDER BY Count DESC")
display(crimeCategory)

category,Count
LARCENY/THEFT,477975
OTHER OFFENSES,303027
NON-CRIMINAL,236937
ASSAULT,191384
VEHICLE THEFT,126228
DRUG/NARCOTIC,117875
VANDALISM,114718
WARRANTS,99821
BURGLARY,91067
SUSPICIOUS OCC,79740


In [11]:
# important hints: 
## first step: spark df or sql to compute the statisitc result 
## second step: export your result to a pandas dataframe. 

crimes_pd_df = crimeCategory.toPandas()

# Spark does not support this function, please refer https://matplotlib.org/ for visuliation. You need to use display to show the figure in the databricks community. 

display(crimes_pd_df)

category,Count
LARCENY/THEFT,477975
OTHER OFFENSES,303027
NON-CRIMINAL,236937
ASSAULT,191384
VEHICLE THEFT,126228
DRUG/NARCOTIC,117875
VANDALISM,114718
WARRANTS,99821
BURGLARY,91067
SUSPICIOUS OCC,79740


#### Q2 question (OLAP)
Counts the number of crimes for different district, and visualize your results

In [13]:
DistrictCrime = spark.sql("SELECT  PdDistrict, COUNT(*) AS Count FROM sf_crime GROUP BY PdDistrict ORDER BY Count DESC")
# crimeCategory = spark.sql("SELECT  category, COUNT(*) AS Count FROM sf_crime GROUP BY category ORDER BY Count DESC")
display(DistrictCrime)

PdDistrict,Count
SOUTHERN,394234
MISSION,293072
NORTHERN,269229
CENTRAL,223962
BAYVIEW,210729
TENDERLOIN,189034
INGLESIDE,186645
TARAVAL,158710
PARK,121070
RICHMOND,114267


#### Q3 question (OLAP)
Count the number of crimes each "Sunday" at "SF downtown".   
hint 1: SF downtown is defiend  via the range of spatial location. For example, you can use a rectangle to define the SF downtown, or you can define a cicle with center as well. Thus, you need to write your own UDF function to filter data which are located inside certain spatial range. You can follow the example here: https://changhsinlee.com/pyspark-udf/

hint 2: SF downtown 物理范围可以是 rectangle a < x < b  and c < y < d. thus, San Francisco Latitude and longitude coordinates are: 37.773972, -122.431297. X and Y represents each. So we assume SF downtown spacial range: X (-122.4213,-122.4313), Y(37.7540,37.7740). 也可以是中心一个圈，距离小于多少算做downtown

In [15]:
SF_DOWNTOWN_CRIME=spark.sql("""
                            with sunday_crime as(
                            select substring(Date,1,5) as date,
                            substring(Date,7,10) as year
                            from sf_crime
                            where (X<=-122.4213 
                            and X>=-122.4313
                            and Y >=37.7540 
                            and Y<=37.7740
                            and DayOfWeek='Sunday')
                            )
                            
                            select date,year,count(*) as count
                            from sunday_crime
                            group by year,date
                            order by year,date
                            """)
display(SF_DOWNTOWN_CRIME)



date,year,count
01/05,2003,14
01/12,2003,20
01/19,2003,17
01/26,2003,13
02/02,2003,14
02/09,2003,22
02/16,2003,13
02/23,2003,14
03/02,2003,16
03/09,2003,8


#### Q4 question (OLAP)
Analysis the number of crime in each month of 2015, 2016, 2017, 2018. Then, give your insights for the output results. What is the business impact for your result?

In [17]:
CRIME_reason1=spark.sql("""
                            with month_crime as(
                            select Date,
                                     substring(Date,1,2) as month,
                                     substring(Date,7,10) as year
                            
                            from sf_crime
                            where Category in('VEHICLE THEFT','LARCENY/THEFT','BURGLARY')
                            )
                            
                            select month,year,count(*) as count
                            from month_crime
                            where year in('2015','2016','2017','2018')
                           
                            
                            group by year,month
                            order by year,month
                            """)


display(CRIME_reason1)


month,year,count
01,2015,4803
02,2015,4304
03,2015,4947
04,2015,4694
05,2015,5227
06,2015,4960
07,2015,4995
08,2015,4810
09,2015,4514
10,2015,4429


even though the data for 2018's criminal events only showed two months, but it showed abvious downward trend . we can estimate that the overall security in SF is better than previous 4 years. 
It is good for the southern/northen part districts where  have  high criminal rates. Since more people can chose to move to there instead of only have choices for western or eastern parts, which are safe but  high rental fee. It is good for local hoptel booking and house property industry.

In [19]:
CRIME_reason2=spark.sql("""
                            with month_crime as(
                            select Date,
                                     substring(Date,1,2) as month,
                                     substring(Date,7,10) as year
                            
                            from sf_crime
                            where Category ='DRUG/NARCOTIC'
                            )
                            
                            select month,year,count(*) as count
                            from month_crime
                            where year in('2015','2016','2017','2018')
                           
                            
                            group by year,month
                            order by year,month
                            """)
display(CRIME_reason2)

month,year,count
01,2015,404
02,2015,306
03,2015,399
04,2015,306
05,2015,296
06,2015,296
07,2015,293
08,2015,375
09,2015,368
10,2015,392


In [20]:
%sql
select Category,count(*)as count
from sf_crime
group by Category


Category,count
FRAUD,41348
SUICIDE,1291
LIQUOR LAWS,3941
SECONDARY CODES,22378
FAMILY OFFENSES,926
MISSING PERSON,44268
OTHER OFFENSES,303027
DRIVING UNDER THE INFLUENCE,5652
WARRANTS,99821
ARSON,3875


#### Q5 question (OLAP)
Analysis the number of crime w.r.t the hour in certian day like 2015/12/15, 2016/12/15, 2017/12/15. Then, give your travel suggestion to visit SF.

In [22]:
date = '12/15/2015'

a = """
  select substring(Time,1,2) as Hour,
  count(*) as Count
  from sf_crime
  
  where date = '{0}' and Category in ('LARCENY/THEFT','DRUG/NARCOTIC','ROBBERY')
  group by Hour
  order by Hour
  
"""
Q5_result=spark.sql(a.format(date))
display(Q5_result)

Hour,Count
00,4
01,1
03,1
04,2
07,3
08,4
09,3
10,9
11,7
12,3


In [23]:
date = '12/15/2015'

a = """
  select substring(Time,1,2) as Hour,
  count(*) as Count
  from sf_crime
  
  where date in ('{0}')
  group by Hour
  order by Hour
  
"""
Q5_result=spark.sql(a.format(date))

# q5_result = spark.sql("""
#                       select substring(Time,1,2) as Hour,
#                       count(*) as Count
#                       from sf_crime
#                       group by Hour
#                       order by Hour
#                       """)
# display(q5_result)
display(Q5_result)





Hour,Count
00,15
01,6
02,5
03,4
04,10
05,3
06,4
07,8
08,12
09,10


we can see that the crimes happen frequently during afternoon, especially during 12 to 18pm. I choose several criminal catrgories, and I found that theft,drug,robery are top three reasons. I thinnk it is because during this time, most people are out of, so for criminals, the overall amount of subjects is also increse.

I think if you hang out along, especially in southern or northern part, it is better not between 12-17pm, because during this time period has high-frequent criminal events.

#### Q6 question (OLAP)
(1) Step1: Find out the top-3 danger disrict  
(2) Step2: find out the crime event w.r.t category and time (hour) from the result of step 1  
(3) give your advice to distribute the police based on your analysis results.

In [27]:
Q6_question=spark.sql("""
                      select PdDistrict,
                      count(*) as count
                      from sf_crime
                      group by PdDistrict
                      order by count DESC
                     
                      """)
display(Q6_question)

PdDistrict,count
SOUTHERN,394234
MISSION,293072
NORTHERN,269229
CENTRAL,223962
BAYVIEW,210729
TENDERLOIN,189034
INGLESIDE,186645
TARAVAL,158710
PARK,121070
RICHMOND,114267


the top 3 crime event places are SOUTHERN,MISSION,NORTHERN

In [29]:
Q6_step2=spark.sql(""" 
         
          select count(Category), 
          substring(Time,1,2) as Hour
          from sf_crime
          where PdDistrict = 'SOUTHERN'
          group by Hour
          order by Hour
        
         
          
""")
display(Q6_step2)

count(Category),Hour
19957,00
11114,01
9232,02
5666,03
3977,04
3420,05
5601,06
9231,07
14158,08
16472,09


In [30]:
Q6_step2_inMission=spark.sql(""" 
         
          select Category, Resolution,
          substring(Time,1,2) as Hour
          from sf_crime
          where PdDistrict = 'SOUTHERN'
         
          
""")
display(Q6_step2_inMission)

Category,Resolution,Hour
LARCENY/THEFT,NONE,04
ASSAULT,DISTRICT ATTORNEY REFUSES TO PROSECUTE,15
WARRANTS,"ARREST, BOOKED",16
OTHER OFFENSES,NONE,19
LARCENY/THEFT,NONE,09
LARCENY/THEFT,"ARREST, BOOKED",20
NON-CRIMINAL,NONE,21
OTHER OFFENSES,"ARREST, BOOKED",15
SUSPICIOUS OCC,NONE,22
VEHICLE THEFT,"ARREST, BOOKED",12


From table, 
1.I find that most of the time if the events happend at night especially after 12am,most of the resolution are None. So it is better to assign more policemen working. 
2.Even in the daytime, the resolution for LARCENY/THEFT still shows None. I think it is better to install more police cameras or warners on streets, for more detailed evidences and it is convenient to arrest robbers.

In [32]:
Q6_step2=spark.sql(""" 
         
          select count(Category), 
          substring(Time,1,2) as Hour
          from sf_crime
          where PdDistrict = 'MISSION'
          group by Hour
          order by Hour
        
         
          
""")
display(Q6_step2)

count(Category),Hour
16517,00
10961,01
8825,02
5564,03
3868,04
3072,05
4742,06
7204,07
10355,08
11132,09


the same regulation with SOUTHERN district, so the local places should assign more policemen espeically during afternoon to night and install more cameras.

In [34]:
Q6_step2_inMission=spark.sql(""" 
         
          select Category, Resolution,
          substring(Time,1,2) as Hour
          from sf_crime
          where PdDistrict = 'MISSION'
         
          
""")
display(Q6_step2_inMission)

Category,Resolution,Hour
LARCENY/THEFT,NONE,13
ASSAULT,NONE,17
BURGLARY,NONE,08
LARCENY/THEFT,"ARREST, BOOKED",12
"SEX OFFENSES, FORCIBLE",NONE,01
LARCENY/THEFT,NONE,20
VANDALISM,NONE,19
OTHER OFFENSES,NONE,18
LARCENY/THEFT,NONE,21
LARCENY/THEFT,"ARREST, CITED",19


In [35]:
Q6_step2_inNORTHERN =spark.sql(""" 
         
          select count(Category), 
          substring(Time,1,2) as Hour
          from sf_crime
          where PdDistrict = 'NORTHERN'
          group by Hour
          order by Hour
        
         
          
         
          
""")
display(Q6_step2_inNORTHERN)

count(Category),Hour
14348,00
9241,01
7732,02
5402,03
3689,04
3265,05
4180,06
6125,07
9365,08
10047,09


#### Q7 question (OLAP)
For different category of crime, find the percentage of resolution. Based on the output, give your hints to adjust the policy.

In [38]:
Q7_question=spark.sql("""
create temp view Category_resolution as
select category, resolution, count(*) as Number_resloution 
from sf_crime 
group by category, resolution
order by category 
""")
display(Q7_question)

In [39]:
Q7_question1=spark.sql("""
create temp view Categoryallresolution as
select Category,
count(*) as overall_numofresolution
from sf_crime 
group by category
order by Category

""")
display(Q7_question1)

In [40]:
Q7_finalquestion=spark.sql("""
create temp view percentage as
select distinct sf_crime.Category, sf_crime.Resolution, Number_resloution / overall_numofresolution as percentage
from (sf_crime left join Category_resolution on sf_crime.Category = Category_resolution.Category and sf_crime.Resolution = Category_resolution.Resolution)
left join Categoryallresolution on sf_crime.Category=Categoryallresolution.Category
order by Category,Resolution

""")
display(Q7_finalquestion)

In [41]:
q7 = spark.sql("""
               select Resolution, Percentage
               from percentage
               where category = 'ASSAULT'
               order by Percentage desc
               """)
display(q7)


Resolution,Percentage
NONE,0.5943391297078126
"ARREST, BOOKED",0.32818312920620324
"ARREST, CITED",0.03750052250971868
DISTRICT ATTORNEY REFUSES TO PROSECUTE,0.013647953851941647
COMPLAINANT REFUSES TO PROSECUTE,0.011108556619153116
UNFOUNDED,0.006661998913179785
EXCEPTIONAL CLEARANCE,0.0038456715294904486
PSYCHOPATHIC CASE,0.0022415666931404924
NOT PROSECUTED,0.0011442962839108806
PROSECUTED BY OUTSIDE AGENCY,7.315136061530745E-4


In [42]:
q8 = spark.sql("""
               select Resolution, Percentage
               from percentage
               where category = 'LARCENY/THEFT'
               order by Percentage desc
               """)
display(q8)

Resolution,Percentage
NONE,0.9162131910664784
"ARREST, BOOKED",0.052588524504419684
"ARREST, CITED",0.021890266227313143
NOT PROSECUTED,0.0033474554108478475
UNFOUNDED,0.0025545269104032637
COMPLAINANT REFUSES TO PROSECUTE,9.854071865683352E-4
DISTRICT ATTORNEY REFUSES TO PROSECUTE,9.770385480412154E-4
EXCEPTIONAL CLEARANCE,9.247345572467179E-4
PROSECUTED BY OUTSIDE AGENCY,3.2637690255766513E-4
PSYCHOPATHIC CASE,9.833150269365553E-5


### Conclusion. 
Use four sentences to summary your work. Like what you have done, how to do it, what the techinical steps, what is your business impact. 
More details are appreciated. You can think about this a report for your manager. Then, you need to use this experience to prove that you have strong background on big  data analysis.  
Point 1:  what is your story ? and why you do this work ?   
Point 2:  how can you do it ?  keywords: Spark, Spark SQL, Dataframe, Data clean, Data visulization, Data size, clustering, OLAP,   
Point 3:  what do you learn from the data ?  keywords: crime, trend, advising, conclusion, runtime

I want to book an apartment in SF. The most important thing I need to consider is safe. So, I need to find the district that has the best security, and I need to figure out during what time in a day is pretty safe to hang out. I choose SF Police Department Incident Reports as starting point to solve all my questions.
First I transfer the data into Spark sql. I select district and category as keywords and rank the secruity of different districts. It helps me to filter the southern and northen parts, since the reslut shows that there have     the most frequent criminial events.
Second, I choose time, district and category as keywords. Through that, I can know during which time in a day always happend criminal events. Knowing about this information, I can choose the pretty safty time if i have to hang out alone.
Third, through data visulization, I clearly know the local police resolutiuon situation. I find that over 80% cases are none. I estimate that, maybe the public infrastructure are not well establishged, the street cameras haven't been installed at some streets, so the police do not have evidences or cannot trace criminals in time. Besides, because the insufficient police, they can only put efforts on some big cases. 
So, after all the research above, I may choose richmond district, which has the lowest crime rates. But I will always check the lastest information in southern or northen parts, because the data showed that the local situation turned better than before.

### Optional part: Clustering
You can apply Spark ML custering algorithm to cluster the spatial data, then visualize the clustering results. Do not do this until you understand Spark ML, we would like to cover this in the DS track.

### Optional part: Time series analysis
This part is not based on Spark, and only based on Pandas Time Series package.   
Note: I am not familiar with time series model, please refer the ARIMA model introduced by other teacher.   
process:  
1.visualize time series  
2.plot ACF and find optimal parameter  
3.Train ARIMA  
4.Prediction 

Refer:   
https://zhuanlan.zhihu.com/p/35282988  
https://zhuanlan.zhihu.com/p/35128342  
https://www.statsmodels.org/dev/examples/notebooks/generated/tsa_arma_0.html  
https://www.howtoing.com/a-guide-to-time-series-forecasting-with-arima-in-python-3  
https://www.joinquant.com/post/9576?tag=algorithm  
https://blog.csdn.net/u012052268/article/details/79452244

#### How to implement a inner join/ left join/right join via Python? 选做哈

For example 

Array 1: [[1, "mike", "f"], [2, "jason", "f"], [3, "robin", "m"], ...] 

Array 2: [[1, "95051", "CS"], [2, "95001", "DS"], [6, "robin", "DE"], ...]